In [2]:
import tensorflow as tf

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

2024-04-25 08:33:22.038029: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 08:33:22.155463: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 08:33:22.155496: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 08:33:22.155884: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 08:33:22.211717: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 08:33:22.213112: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
clients = [1, 2, 3]

In [4]:
import tensorflow as tf
import numpy as np

# Load the MNIST dataset
(x_train_all, y_train_all), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

In [5]:
from sklearn.model_selection import train_test_split

# assume X is your feature data and y is your target data
X_train, X_test, y_train, y_test = train_test_split(
    x_train_all, y_train_all, test_size=0.2, random_state=42
)

# split data into n parts
n_parts = len(clients)
part_size = len(X_train) // n_parts
dataset_parts = []
for i in range(n_parts):
    start = i * part_size
    end = (i + 1) * part_size
    X_part = X_train[start:end]
    y_part = y_train[start:end]
    dataset_parts.append((X_part, y_part))

In [6]:
def model_init():
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [7]:
models = []
for _ in range(len(clients)):
    models.append(model_init())

2024-04-25 08:33:26.146402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 08:33:26.248892: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
import numpy as np
from Pyfhel import Pyfhel

HE = Pyfhel()  # Creating empty Pyfhel object
ckks_params = {
    "scheme": "CKKS",  # can also be 'ckks'
    "n": 2**14,  # Polynomial modulus degree. For CKKS, n/2 values can be
    #  encoded in a single ciphertext.
    #  Typ. 2^D for D in [10, 15]
    "scale": 2**30,  # All the encodings will use it for float->fixed point
    #  conversion: x_fix = round(x_float * scale)
    #  You can use this as default scale or use a different
    #  scale on each operation (set in HE.encryptFrac)
    "qi_sizes": [60, 30, 30, 30, 60],  # Number of bits of each prime in the chain.
    # Intermediate values should be  close to log2(scale)
    # for each operation, to have small rounding errors.
}
HE.contextGen(**ckks_params)  # Generate context for ckks scheme
HE.keyGen()  # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()

In [14]:
shapedims = [l.shape for l in models[0].get_weights()]
print(shapedims)

[(784, 128), (128,), (128, 10), (10,)]


In [ ]:
# cwts[model][layer][chunk]

In [34]:
cwts = []
for model, dataset in zip(models, dataset_parts):
    model.fit(dataset[0], dataset[1], epochs=5)
    # model.get_weights()
    wtarray = model.get_weights()
    # wts.append(model.get_weights())
    cwt = []
    for layer in wtarray:
        flat_array = layer.astype(np.float64).flatten()  

        chunks = np.array_split(flat_array, (len(flat_array) + 2**10 - 1) // 2**10)
        clayer = []
        print(len(chunks))
        for chunk in chunks:
            ptxt = HE.encodeFrac(chunk)
            ctxt = HE.encryptPtxt(ptxt)
            clayer.append(ctxt)
        print(len(clayer))
        cwt.append(clayer)
    cwts.append(cwt)

Epoch 1/5
500/500 [==============================] - 1s 1ms/step - loss: 0.2027 - accuracy: 0.9524
Epoch 2/5
500/500 [==============================] - 1s 1ms/step - loss: 0.2158 - accuracy: 0.9493
Epoch 3/5
500/500 [==============================] - 1s 1ms/step - loss: 0.2147 - accuracy: 0.9514
Epoch 4/5
500/500 [==============================] - 1s 1ms/step - loss: 0.1697 - accuracy: 0.9578
Epoch 5/5
500/500 [==============================] - 1s 1ms/step - loss: 0.1762 - accuracy: 0.9567
98
98
1
1
2
2
1
1
Epoch 1/5
500/500 [==============================] - 1s 1ms/step - loss: 0.2172 - accuracy: 0.9505
Epoch 2/5
500/500 [==============================] - 1s 1ms/step - loss: 0.2001 - accuracy: 0.9509
Epoch 3/5
500/500 [==============================] - 1s 1ms/step - loss: 0.1802 - accuracy: 0.9560
Epoch 4/5
500/500 [==============================] - 1s 1ms/step - loss: 0.1688 - accuracy: 0.9567
Epoch 5/5
500/500 [==============================] - 1s 1ms/step - loss: 0.1893 - accuracy:

In [51]:
res = []
for j in range(len(cwts[0])):#for layers
    layer = []
    for k in range(len(cwts[0][j])):# for chunks
        tmp = cwts[0][j][k].copy()
        for i in range(1 , len(cwts)): #for clients
            tmp = tmp + cwts[i][j][k]
        tmp = tmp/len(cwts)
        layer.append(tmp)
    res.append(layer)

In [54]:
res[1]

[<Pyfhel Ciphertext at 0x7070fae89cc0, scheme=ckks, size=2/2, scale_bits=60, mod_level=1>]

In [11]:
model = model_init()

In [12]:
wts = model.get_weights()

In [13]:
print(len(wts[0][0] ) , wts[0][0])

128 [ 0.04612084  0.00621976  0.02093226  0.07553466 -0.06475656 -0.02738096
  0.03957256  0.07681689 -0.03073152  0.000495    0.05674433  0.06710587
  0.04723854 -0.02999434  0.06728352 -0.07080525 -0.05492576  0.03444795
  0.07058109  0.03419232 -0.02801988 -0.07718219  0.0658543   0.06254558
 -0.01010962 -0.06199992  0.07691651  0.06381496 -0.03306201  0.04707106
 -0.00812148 -0.0304095  -0.05652771 -0.05179098  0.02299691  0.02021011
 -0.01430169  0.01682939  0.03599281  0.04883536 -0.00994213 -0.03305173
  0.02341475  0.07644627  0.05962389 -0.02852966  0.04409241 -0.03527503
 -0.06908529  0.00908931  0.03092042 -0.06849469 -0.07471936 -0.0003303
  0.01822687  0.06146234  0.00696314  0.05320413  0.02402794 -0.04019794
  0.0522406   0.0513887   0.02972488 -0.02117474 -0.01950202 -0.01380565
  0.01530191 -0.0444936   0.06150436 -0.01408558 -0.03226737  0.05069966
  0.00220674  0.00228312  0.07616719  0.07787687 -0.04631416  0.05078193
 -0.01162528  0.04441049 -0.05280068 -0.05408106